<a href="https://colab.research.google.com/github/yeoun9/torchpm/blob/main/examples/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install git+https://github.com/yeoun9/torchpm.git
! pip install sympytorch

In [1]:
from torchpm import *
from torchpm.data import CSVDataset
from torchpm.parameter import *
import torch as tc
import numpy as np
import matplotlib.pyplot as plt

# Model Building

In [2]:
dataset_file_path = 'https://raw.githubusercontent.com/yeoun9/torchpm/main/examples/THEO_cov_searching.csv'
dataset_np = np.loadtxt(dataset_file_path, delimiter=',', dtype=np.float32, skiprows=1)
device = tc.device("cuda:0" if tc.cuda.is_available() else "cpu")
column_names = ['ID', 'AMT', 'TIME', 'DV', 'CMT', "MDV", "RATE", 'BWT', 'fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5']
dataset = CSVDataset(dataset_np, column_names, device)

class BasementModel(predfunction.PredictionFunctionByTime) :
    
    def _set_estimated_parameters(self):
        self.theta_0 = Theta(0., 1.5, 10.)
        self.theta_1 = Theta(0., 30., 100.)
        self.theta_2 = Theta(0, 0.08, 1)

        self.eta_0 = Eta()
        self.eta_1 = Eta()
        self.eta_2 = Eta()

        self.eps_0 = Eps()
        self.eps_1 = Eps()

        self.gut_model = linearode.Comp1GutModelFunction()
    
    def _calculate_parameters(self, covariates):
        covariates['k_a'] = self.theta_0()*tc.exp(self.eta_0())
        covariates['v'] = self.theta_1()*tc.exp(self.eta_1())#*para['BWT']/70
        covariates['k_e'] = self.theta_2()*tc.exp(self.eta_2())
        covariates['AMT'] = tc.tensor(320., device=self.dataset.device)

    def _calculate_preds(self, t, p):
        dose = p['AMT'][0]
        k_a = p['k_a']
        v = p['v']
        k_e = p['k_e']
        comps = self.gut_model(t, k_a, k_e, dose)
        return comps[1]/v
        
    def _calculate_error(self, y_pred, p):
        p['v_v'] = p['v'] 
        return y_pred +  y_pred * self.eps_0() + self.eps_1()

dependent_parameter_names = ['k_a', 'v', 'k_e']
dependent_parameter_initial_values = [[0,1.4901,2],[30,32.4667,34],[0,0.08,0.1]]
independent_parameter_names = ['BWT', 'fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5']

searcher = covariate.DeepCovariateSearching(dataset=dataset,
                                BaseModel=BasementModel,
                                dependent_parameter_names=dependent_parameter_names,
                                independent_parameter_names=independent_parameter_names,
                                dependent_parameter_initial_values=dependent_parameter_initial_values,
                                eps_names=['eps_0', 'eps_1'],
                                omega = Omega([0.4397,
                                                0.0575,  0.0198, 
                                                -0.0069,  0.0116,  0.0205], False, requires_grads=True),
                                sigma = Sigma([[0.0177], [0.0762]], [True, True], requires_grads=[True, True]))
print(searcher.run())

running_time :  1.5699994564056396 	 total_loss: tensor(490.1509, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  2.176999568939209 	 total_loss: tensor(403.7401, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  2.7679991722106934 	 total_loss: tensor(343.5059, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  3.3489980697631836 	 total_loss: tensor(267.9600, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  3.9519994258880615 	 total_loss: tensor(243.7507, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  4.5439982414245605 	 total_loss: tensor(202.4627, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  5.140998840332031 	 total_loss: tensor(168.2006, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  5.735997676849365 	 total_loss: tensor(137.0659, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  6.335001707077026 	 total_loss: tensor(131.7247, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  6.926997423171997 	 total

FOCEInter(
  (pred_function_module): BasementModel(
    (theta_0): Theta()
    (theta_1): Theta()
    (theta_2): Theta()
    (eta_0): Eta(
      (parameter_values): ParameterDict(
          (1): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (2): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (3): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (4): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (5): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (6): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (7): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (8): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (9): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (10): Parameter containing: [torch.cuda.FloatTensor of size  (GPU 0)]
          (11): Parameter con